### Initialise data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.lines as mlines
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic
from skimage import transform as tf
from skimage import io, measure
from IPython.display import display
import scipy as scipy
import peakutils

import os
if os.name == 'posix':
    %matplotlib osx

#Load in DIC maps
DicFilePath = "newdata/"
DicMap = hrdic.Map(DicFilePath, "irr_003.TXT")
DicMap.setPatternPath("irr_000_8bin.jpg", 2)
DicMap.setCrop(xMin=150,xMax=293,yMin=115,yMax=214)

DicMap2 = hrdic.Map(DicFilePath, "nonirr_003.TXT")
DicMap2.setPatternPath("nonirr_000_8bin.jpg", 2)
DicMap2.setCrop(xMin=131,xMax=166,yMin=100,yMax=99)

pixelinm=78.125e-9

#Load in EBSD map and calculate misorientation
EbsdMap = ebsd.Map("newdata/irr_02_cleaned", "hexagonal")
EbsdMap.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 3.6)
EbsdMap.boundaries = scipy.misc.imread('newdata/irr_boundaries.bmp').astype(int)
EbsdMap.boundaries[EbsdMap.boundaries == 0] = -1
EbsdMap.boundaries[EbsdMap.boundaries == 255] = 0
EbsdMap.findGrains(minGrainSize= 100)
EbsdMap.calcGrainMisOri(calcAxis = False)
EbsdMap.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))

EbsdMap2 = ebsd.Map("newdata/nonirr_02_cleaned", "hexagonal")
EbsdMap2.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap2.buildQuatArray()
EbsdMap2.findBoundaries(boundDef = 3.6)
EbsdMap2.boundaries = scipy.misc.imread('newdata/nonirr_boundaries.bmp').astype(int)
EbsdMap2.boundaries[EbsdMap2.boundaries == 0] = -1
EbsdMap2.boundaries[EbsdMap2.boundaries == 255] = 0
EbsdMap2.findGrains(minGrainSize= 100)
EbsdMap2.calcGrainMisOri(calcAxis = False)
EbsdMap2.calcAverageGrainSchmidFactors(loadVector=np.array([1, 0, 0]))

DicMap2.homogPoints = np.array((((326, 377), (118 , 540),(1166, 955), (1108, 1012), (142, 1042), (252, 1092), (64, 766), (68,200), 
                                 (710, 936), (580, 1128), (1026, 484), (1204, 306), (1166, 600), (1044,  852), (888, 1101), (636, 609),
                                 (485, 890), (486, 628), (700, 276), (414, 219), (874, 670), (863, 59), (226, 44), (552, 28))))
EbsdMap2.homogPoints = np.array(((167, 232), (80, 286), (484, 469), (462, 489), (79, 472), (121, 494), (54, 368), (67, 160), 
                                 (305, 446), (253, 516), (440, 289), (513, 229), (491, 337), (438, 428), (373, 519), (281, 325),
                                 (218, 423), (226, 327), (317, 204), (203, 174), (378, 353), (385, 128), (135, 106), (262, 106)))
DicMap.homogPoints = np.array(( (430, 219), (107, 1082), (111, 164), (547, 1106), (1164, 1108),
                               (1410, 887), (1424, 608), (1290, 180), (846, 236), (814, 36),
                               (462, 58), (1000, 620), (43, 652), (206, 498), (476, 723),
                               (582, 747), (850, 1088), (1183, 764), (686, 428), (668, 538),
                               (854, 851) ))
EbsdMap.homogPoints = np.array(( (202, 141), (57, 434), (78, 118), (228, 450), (471, 461), (570, 388),
                                (582, 290), (539, 144), (365, 155), (358, 87), (217, 90), (415, 288),
                                (40, 284), (107, 233), (209, 315), (247, 325), (347, 447), (484, 341),
                                (299, 215), (289, 253), (346, 357) ))

DicMap.homogPoints[:,:1] -= 50
DicMap.homogPoints[:,1:2] -= 15

DicMap.linkEbsdMap(EbsdMap)
DicMap.findGrains(minGrainSize=100)

DicMap2.linkEbsdMap(EbsdMap2)
DicMap2.findGrains(minGrainSize=100)

Loaded DaVis 8.1.5 data (dimensions: 1659 x 1481 pixels, sub-window size: 16 x 16 pixels)
Loaded DaVis 8.1.5 data (dimensions: 1513 x 1351 pixels, sub-window size: 16 x 16 pixels)
Loaded EBSD data (dimensions: 721 x 605 pixels, step size: 0.2 um)
Loaded EBSD data (dimensions: 696 x 599 pixels, step size: 0.2 um)


### Export and set boundaries

In [ ]:
#EXPORT BOUNDARY TO FILE
from skimage import morphology as mph
import matplotlib as mpl
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
boundariesImage = -DicMap2.boundaries
boundariesImage2 = mph.binary_dilation(boundariesImage)
plt.imshow(boundariesImage2, cmap=cmap1, vmin=0, vmax=1)

import scipy.misc
scipy.misc.imsave('newdata/nirr_exp.bmp', boundariesImage)

In [ ]:
#DicMap2.homogPoints = np.array((   ))
#EbsdMap2.homogPoints = np.array((   ))

#DicMap2.setHomogPoint(display="pattern")
#EbsdMap2.setHomogPoint()

DicMap2.setHomogPoint()
EbsdMap2.setHomogPoint()

### EBSD Misori plot

In [ ]:
EbsdMap.calcGrainMisOri(calcAxis = True)
EbsdMap2.calcGrainMisOri(calcAxis = True)

In [ ]:
EbsdMap2.plotMisOriMap(plotGBs=True, boundaryColour='black', vmin=-4, vmax=4,
                      cmap="coolwarm")

### Plot DIC maps

In [65]:
DicMap2.plotMaxShear(plotGBs=True,vmin=0,vmax=0.08,plotColourBar=True,dilateBoundaries=True)

In [ ]:
from matplotlib.colors import PowerNorm
fig, ((ax0)) = plt.subplots(1, 1, figsize=(20, 10))
plt.imshow(DicMap2.crop(DicMap2.max_shear),vmin=0.0, vmax=0.4,cmap='viridis',norm=PowerNorm(gamma=0.4))
plt.colorbar()

In [ ]:
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

selDicMap = DicMap
from matplotlib_scalebar.scalebar import ScaleBar
fig, ((ax0)) = plt.subplots(1, 1, figsize=(6, 4))
plt.imshow(selDicMap.crop(selDicMap.max_shear),vmin=0.0, vmax=0.1,cmap='viridis')
plt.colorbar()
ax0.add_artist(ScaleBar(pixelinm))

ax0.xaxis.set_visible(False)
ax0.yaxis.set_visible(False)

boundariesImage = -selDicMap.boundaries
boundariesImage = mph.binary_dilation(boundariesImage)
ax0.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

In [ ]:
from matplotlib.colors import PowerNorm
vmax=0.4; gamma=0.6;
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
img=ax0.imshow(DicMap.crop(DicMap.max_shear),cmap='viridis',vmin=0,vmax=vmax, norm=PowerNorm(gamma=gamma))
img2=ax1.imshow(DicMap2.crop(DicMap2.max_shear),cmap='viridis',vmin=0,vmax=vmax, norm=PowerNorm(gamma=gamma))
plt.colorbar(img, ax=ax0, label="Effective shear strain (%)")
plt.colorbar(img, ax=ax1, label="Effective shear strain (%)")

In [ ]:
vmin=-10; vmax=10; cmap='coolwarm';

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2)

img=ax0.imshow(DicMap.crop(DicMap.f11)*100-100,cmap=cmap,vmin=vmin,vmax=vmax)
img2=ax3.imshow(DicMap2.crop(DicMap2.f22)*100-100,cmap=cmap,vmin=vmin,vmax=vmax)
img3=ax2.imshow(DicMap2.crop(DicMap2.f21)*100,cmap=cmap,vmin=vmin,vmax=vmax)
img4=ax1.imshow(DicMap2.crop(DicMap2.f12)*100,cmap=cmap,vmin=vmin,vmax=vmax)

ax0.xaxis.set_visible(False), ax0.yaxis.set_visible(False)
ax1.xaxis.set_visible(False), ax1.yaxis.set_visible(False)
ax2.xaxis.set_visible(False), ax2.yaxis.set_visible(False)
ax3.xaxis.set_visible(False), ax3.yaxis.set_visible(False)

fig.tight_layout()

### Plot EBSD Maps

In [ ]:
EbsdMap2.plotMisOriMap(plotGBs=True,vmin=0,vmax=6)

In [ ]:
#### DOES NOT WORK .... YET
import matplotlib as mpl
from skimage import morphology as mph

self=DicMap2
fig = plt.figure(frameon=False)
schmidlist=[]

ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
self.misOri = np.ones([self.yDim, self.xDim])
avschmid = np.zeros([self.yDim, self.xDim])
for grain in self.grainList:
    for coord, misOri in zip(grain.coordList, grain.misOriList):
        self.misOri[coord[1], coord[0]] = misOri
ax.imshow(np.arccos(self.misOri) * 360 / np.pi,interpolation='none', cmap='gray', vmin=0, vmax=0.5)

boundariesImage = -self.boundaries
boundariesImage = mph.binary_dilation(boundariesImage)
ax.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

fig.savefig('fig2.png',dpi=600, bbox_inches='tight', pad_inches=0)
print('Average prismatic schmid: {0:.3f} +- {1:.3f}'.format(np.mean(schmidlist),np.std(schmidlist)))

### Stats

In [94]:
DicMap.printStatsTable(percentiles=['Min', 1, 5, 'Mean', 95, 99.99, 'Max'], components=['f11','mss'])
DicMap2.printStatsTable(percentiles=['Min', 1, 5, 'Mean', 95, 99.99, 'Max'], components=['f11','mss'])

DicMap (dimensions: 1659 x 1481 pixels, sub-window size: 16 x 16 pixels, number of points: 2456979)

Component	Min	P1	P5	Mean	P95	P99.99	Max	
f11		-24.16	-0.16	0.11	2.04	5.41	61.91	80.51	
mss		0.00	0.09	0.22	1.95	5.16	54.34	75.39	

DicMap2 (dimensions: 1513 x 1351 pixels, sub-window size: 16 x 16 pixels, number of points: 2044063)

Component	Min	P1	P5	Mean	P95	P99.99	Max	
f11		-12.74	0.20	0.68	2.59	4.96	27.89	39.95	
mss		0.00	0.30	0.65	2.39	4.50	26.03	52.37	



In [66]:
print((100-99.99999)*DicMap.xDim*DicMap.yDim)

14.0083200044


In [103]:
plt.imshow(DicMap2.crop(DicMap2.max_shear)>0.3)

In [55]:
irr_crop= DicMap.crop(DicMap.max_shear)
irr_crop= DicMap.crop(DicMap.max_shear) / np.mean(DicMap.crop(DicMap.max_shear))
ni_crop= DicMap2.crop(DicMap2.max_shear)
ni_crop= DicMap2.crop(DicMap2.max_shear) / np.mean(DicMap2.crop(DicMap2.max_shear))

hist, bins = np.histogram(irr_crop, bins=100,range=[0,10],density=True)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
#plt.show()
np.savetxt('irr.csv', (hist), delimiter=',')

hist, bins = np.histogram(ni_crop, bins=100,range=[0,10],density=True)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
#plt.show()
np.savetxt('nonirr.csv', (hist), delimiter='\n')

#### normal

In [ ]:
irr_crop= DicMap.crop(DicMap.max_shear)
irr_crop= DicMap.crop(DicMap.max_shear) / np.mean(DicMap.crop(DicMap.max_shear))
ni_crop= DicMap2.crop(DicMap2.max_shear)
ni_crop= DicMap2.crop(DicMap2.max_shear) / np.mean(DicMap2.crop(DicMap2.max_shear))

irr2=[item for sublist in irr_crop for item in sublist]
nirr2=[item for sublist in ni_crop for item in sublist]

f, (ax1) = plt.subplots(1)
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

counts, bins, bars = ax1.hist(irr2, bins=np.linspace(0,50,800), color='r', histtype='step')
plt.gca().set_xlabel("Effective shear strain (multiples of average)")
plt.gca().set_ylabel("Frequency")

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('irr.csv', (center, counts), delimiter='\n')

counts2, bins, bars = ax1.hist(nirr2, bins=np.linspace(0,50,800), color='g', histtype='step')
#ax1.set_xscale("log", nonposx='clip')
#ax1.set_yscale("log", nonposy='clip')
ax1.set_xlim(1e-2,5)
#ax1.set_ylim(1e-6,1e1)

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('nonirr.csv', (center, counts2), delimiter='\n')

plt.show()

#### logy

In [ ]:
irr_crop= DicMap.crop(DicMap.max_shear)
irr_crop= DicMap.crop(DicMap.max_shear) / np.mean(DicMap.crop(DicMap.max_shear))
ni_crop= DicMap2.crop(DicMap2.max_shear)
ni_crop= DicMap2.crop(DicMap2.max_shear) / np.mean(DicMap2.crop(DicMap2.max_shear))

irr2=[item for sublist in irr_crop for item in sublist]
nirr2=[item for sublist in ni_crop for item in sublist]

counts, bins, bars = plt.hist(irr2, bins=np.linspace(0,40,100),histtype='step',normed='true')
plt.gca().set_yscale("log")
plt.gca().set_xlabel("Effective shear strain (multiples of average)")
plt.gca().set_ylabel("Frequency")

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('irr.csv', (center, counts), delimiter='\n')

counts, bins, bars = plt.hist(nirr2, bins=np.linspace(0,40,100),histtype='step',normed='true')
plt.gca().set_yscale("log")
plt.show()
center = (bins[:-1] + bins[1:]) / 2
np.savetxt('nonirr.csv', (center, counts), delimiter='\n')

#### loglog

In [26]:
irr_crop= DicMap.crop(DicMap.max_shear)
irr_crop= DicMap.crop(DicMap.max_shear) / np.mean(DicMap.crop(DicMap.max_shear))
ni_crop= DicMap2.crop(DicMap2.max_shear)
ni_crop= DicMap2.crop(DicMap2.max_shear) / np.mean(DicMap2.crop(DicMap2.max_shear))

irr2=[item for sublist in irr_crop for item in sublist]
nirr2=[item for sublist in ni_crop for item in sublist]

f, (ax1) = plt.subplots(1, figsize=(6, 4))

counts, bins, bars = ax1.hist(irr2, bins=np.logspace(np.log10(0.001),np.log10(50.0),100), color='r', histtype='step',  label='Irradiated',normed=True)
plt.xlabel("Effective shear strain (multiples of average)", fontsize="12")
plt.ylabel("Frequency", fontsize="12")
center = (bins[:-1] + bins[1:]) / 2
np.savetxt('irr.csv', (center, counts), delimiter='\n')

counts2, bins, bars = ax1.hist(nirr2, bins=np.logspace(np.log10(0.001),np.log10(50.0),100), color='g', histtype='step',  label='Non irradiated',normed=True)
plt.xscale("log", nonposx='clip'); plt.yscale("log", nonposy='clip');
plt.xlim(1e-2,5e1); 
plt.ylim(0,1e2);

center = (bins[:-1] + bins[1:]) / 2
np.savetxt('nonirr.csv', (center, counts2), delimiter='\n')

plt.tick_params(top='on', right='on', which='both', direction='in')
plt.tick_params(axis='y', which='minor')
plt.axvline(1, color='black', linestyle='dashed', linewidth=1)
plt.legend()
f.tight_layout()
plt.show()

f.savefig('histograms.png',dpi=400, pad_inches=0)

In [ ]:
plt.loglog(center,counts,'r.',basex=10,basey=10,)

In [ ]:
gs = gridspec.GridSpec(1, 3)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1],sharex=ax0,sharey=ax0)
ax2=plt.subplot(gs[2],sharex=ax0,sharey=ax0)
ax0.imshow(DicMap.crop(DicMap.max_shear),vmax=0.1)
ax1.imshow(DicMap.crop(DicMap.x_map),vmax=0.01)
ax2.imshow(DicMap.crop(DicMap.y_map),vmax=0.01)

### Plot numbered grains

In [ ]:
DicMap.plotNumberedGrain()

### RDR Calculation

In [ ]:
DicMap2.plotMaxShear(plotGBs=True,vmin=0,vmax=10,plotColourBar=True,dilateBoundaries=False)

In [ ]:
### Install matplotlib_scalebar and pandas
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib as mpl
import pandas as pd
pixelinm=78.125e-9

def runRDR(x0,y0,x1,y1,length=5,cond=10,rcond=0,SelDicMap=DicMap):

    ulist=[]; vlist=[]; gradlist=[]; rsquaredlist=[]; index=[1]; id=[]; sliptraceno=[];

    ### Generate plots
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(12, 6))
    img = ax0.imshow(SelDicMap.crop(SelDicMap.max_shear),vmin=0,vmax=0.08, cmap='viridis')
    ### Set axis limits to only show region of interest
    ax0.set_xlim(np.amin([x0,x1])-50,np.amax([x0,x1])+50)
    ax0.set_ylim(np.amax([y0,y1])+50,np.amin([y0,y1])-50)
    ### Add scale bar, color bar and grain boundary maps
    ax0.add_artist(ScaleBar(pixelinm))
    fig.colorbar(img, ax=ax0,fraction=0.046, pad=0.04, label="Effective shear strain (%)")
    cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
    cmap1._init()
    cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
    boundariesImage = -SelDicMap.boundaries
    ax0.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)
    colors=['r','b','k','g','y','c','m']

    ### For each slip trace line
    for x0,y0,x1,y1 in zip(x0,y0,x1,y1):
        
        ### Plot numbered arrow
        ax0.arrow(x0, y0, x1-x0, y1-y0, head_width=5, head_length=5, fc='k', ec='k', lw=2)
        ax0.annotate(index[0], xy=(x0, y0), xycoords='data', xytext=(5, 5), textcoords='offset points')
        index[0]+=1

        ### Calculate properties for drawn line
        grad = (y1-y0)/(x1-x0)
        invgrad = -1/grad
        profile_length = np.sqrt((y1-y0)**2+(x1-x0)**2)
        num = np.round(1+profile_length*2)

        angle=np.arctan(np.abs(y1-y0)/np.abs(x1-x0))*180/3.141592654
        if grad>0:
            angle=180-angle
            
        print(angle)
        
        ### Calculate positions for each point along slip trace line (x,y)
        x, y = np.linspace(x0, x1, int(num)), np.linspace(y0, y1, int(num))
        x = np.round(x); y=np.round(y);
       
        ### Remove duplicate points
        df = pd.DataFrame({'x':x, 'y':y})
        df = df.drop_duplicates()
        x,y=df['x'].values.tolist(),df['y'].values.tolist()
        
        ### Plot
        ax0.plot(x,y, 'rx',lw=2)

        ### For each point along slip line
        for x,y in zip(x,y):

            ## Calculate positions for each point along lines perpendicular to slip line (xnew,ynew)
            x0new = x+np.sqrt(length/(invgrad**2+1))*np.sign(grad)
            y0new = y-np.sqrt(length/(1/invgrad**2+1))
            x1new = x-np.sqrt(length/(invgrad**2+1))*np.sign(grad)
            y1new = y+np.sqrt(length/(1/invgrad**2+1))
            profile_length=np.sqrt((y1new-y0new)**2+(x1new-x0new)**2)
            num = np.round(profile_length)
            xnew, ynew = np.linspace(x0new, x1new, int(num)), np.linspace(y0new, y1new, int(num))
            xnew, ynew=np.around(xnew).astype(int), np.around(ynew).astype(int)
            df = pd.DataFrame({'x':xnew, 'y':ynew})
            df = df.drop_duplicates()
            xnew,ynew=df['x'].values.tolist(),df['y'].values.tolist()

            ### Plot
            ax0.plot(xnew,ynew, 'rx',lw=0.5)
            ax0.plot(xnew,ynew, '-',lw=0.5)
            
            ### For all points, append u and v to list
            u =[]; v =[];
            for xnew,ynew in zip(xnew,ynew):
                u.append((SelDicMap.crop(SelDicMap.x_map))[ynew,xnew])
                v.append((SelDicMap.crop(SelDicMap.y_map))[ynew,xnew])
            
            ### Take away mean
            u=u-np.mean(u)
            v=v-np.mean(v)
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x=v,y=u)

            ### Append to main arrays (ulist,vlist) (conditional upon u-centered, v-centered and individual rsquared-value)
            if all(cond >= i >= -cond for i in u) and all(cond >= i >= -cond for i in v) and r_value**2 > rcond:
                ulist.extend(u)
                vlist.extend(v)
                gradlist.append(slope)
                rsquaredlist.append(r_value**2)

    ### Linear regression of ucentered against vcentered
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x=vlist,y=ulist)

    ### Generate plots
    ax1.scatter(x=vlist,y=ulist,marker='x', lw=1)
    ax1.plot([np.min(vlist), np.max(vlist)],[slope*np.min(vlist)+intercept,slope*np.max(vlist)+intercept], '-')
    ax1.set_xlabel('v-centered')
    ax1.set_ylabel('u-centered')
    ax1.text(0.95, 0.01, 'Slope = {0:.3f} ± {1:.3f}\nR-squared = {2:.3f}\nn={3}'.format(slope,std_err,r_value**2,len(ulist)),
             verticalalignment='bottom', horizontalalignment='right', transform=ax1.transAxes, fontsize=10);

    ### Print regressions paramaters
    print("Slope {0:.3f} +- {1:.3f}".format(slope,std_err))
    print("R-squared: {0:.3f}".format(r_value**2))
    print("n: {0}".format(len(ulist)))

    ### Find grain
    Grain=SelDicMap.grainList[SelDicMap.grains[y0][x0]-1]
    print("Grain ID : {0}".format(SelDicMap.grains[y0][x0]-1))
    ebsdGrain = Grain.ebsdGrain

    ### Print euler angles for grain
    ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
    ebsdGrain.calcSlipTraces()
    print("Euler angles: ")
    print("",ebsdGrain.refOri.eulerAngles()*180/np.pi, "\n")
    
    ### Print slip systems, directions, schmid, rdr
    RDRs=[]; sts=[]; sfs=[]
    for ssGroup, sfGroup, slipTraceAngle in zip(EbsdMap.slipSystems, ebsdGrain.averageSchmidFactors, ebsdGrain.slipTraceAngles):
        print("Slip Plane Normal {0:s}\tAngle: {1:.1f}".format(ssGroup[0].slipPlaneLabel, slipTraceAngle))
        for ss, sf in zip(ssGroup, sfGroup):
            slipDirSample=ebsdGrain.refOri.conjugate.transformVector(ss.slipDir)
            print("\tSlip Direction {0:s}\tSchmid factor: {1:.3f}\tRDR: {2:.3f}".
                  format(ss.slipDirLabel, sf,-slipDirSample[0]/slipDirSample[1]))
            sfs.append(sf)
            RDRs.append(-slipDirSample[0]/slipDirSample[1])
    print(RDRs)
    xvals=np.zeros(len(RDRs))
    ax2.plot(xvals,RDRs,'o')
    ax2.plot([0],slope,'ro')
    ax2.set_ylim(-5,5)
    ax2.set_xlim(-0.01,0.05)

    sts = ebsdGrain.slipTraceAngles
    
    ### Plot number line labels (only make sense for hexagonal)
    labels=['','','','','','','','','','','','','','','','','','','','','','','','']
    labels[0] = '<a> Ba {0:.1f} {1:.3f} Pr {2:.1f} {3:.3f} Py {4:.1f} {5:.3f} Py {6:.1f} {7:.3f}'.format(sts[0],sfs[0],sts[3],sfs[5],sts[4],sfs[6],sts[5],sfs[9])
    labels[1] = str('<a> Ba {0:.1f} {1:.3f} Pr {2:.1f} {3:.3f} Py {4:.1f} {5:.3f} Py {6:.1f} {7:.3f}'.format(sts[0],sfs[1],sts[1],sfs[3],sts[6],sfs[12],sts[7],sfs[15]))
    labels[2] = str('<a> Ba {0:.1f} {1:.3f} Pr {2:.1f} {3:.3f} Py {4:.1f} {5:.3f} Py {6:.1f} {7:.3f}'.format(sts[0],sfs[2],sts[2],sfs[4],sts[8],sfs[18],sts[9],sfs[21]))
    labels[7] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[4],sfs[7],sts[6],sfs[14]))
    labels[8] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[4],sfs[8],sts[8],sfs[20]))
    labels[11] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[5],sfs[11],sts[7],sfs[17]))
    labels[16] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[7],sfs[16],sts[8],sfs[19]))
    labels[22] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[9],sfs[22],sts[6],sfs[13]))
    labels[23] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[9],sfs[23],sts[5],sfs[10]))

    for i, txt in enumerate(labels):
        ax2.annotate(txt, (xvals[i]+0.002,RDRs[i]-0.05))

    ax2.annotate('{0:.3f}'.format(slope), (-0.009,slope-0.05))

    labels=['','','','','','','','','','','','','','','','','','','','','','','','']
    labels[0] = '{0:.3f}'.format(RDRs[0])
    labels[1] = str('{0:.3f}'.format(RDRs[1]))
    labels[2] = str('{0:.3f}'.format(RDRs[2]))
    labels[7] = str('{0:.3f}'.format(RDRs[7]))
    labels[8] = str('{0:.3f}'.format(RDRs[8]))
    labels[11] = str('{0:.3f}'.format(RDRs[11]))
    labels[16] = str('{0:.3f}'.format(RDRs[16]))
    labels[22] = str('{0:.3f}'.format(RDRs[22]))
    labels[23] = str('{0:.3f}'.format(RDRs[23]))

    for i, txt in enumerate(labels):
        ax2.annotate(txt, (-0.009,RDRs[i]-0.05))

    ### Get rid of whitespace
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.2)

    ### Plot RDR as a function of distance
    #fig2, (ax0) = plt.subplots(1, 1, figsize=(12, 6))
    #plt.plot(gradlist)
    #plt.show()

In [ ]:
# Define the slip trace lines (x0,y0)->(x1,y1)
## Grain 20
x0=np.array([333,316,309,345,352,314,322])
y0=np.array([83,91,105,77,72,126,147])
x1=np.array([428,419,402,431,433,390,367])
y1=np.array([167,182,188,153,143,192,185])

## Grain 63
x0=np.array([505,515,593,559, 476])
y0=np.array([639,667,605,540, 592])
x1=np.array([606,618,627,494, 518])
y1=np.array([537,564,572,609, 548])

## Grain 28
x0=np.array([997,905,1023,1049])
y0=np.array([218,168,268,297])
x1=np.array([916,845,934,967])
y1=np.array([285,220,341,365])

## Grain 103
x0=np.array([174, 154, 141, 135, 110, 186])
y0=np.array([1063, 1042, 1015, 1001, 999, 1076])
x1=np.array([143, 115, 91, 94, 76, 160])
y1=np.array([1104, 1087, 1074, 1049, 1037, 1108])

##Grain 86
x0=np.array([715, 766, 711, 794])
y0=np.array([905, 906, 932, 937])
x1=np.array([745, 819, 781, 829])
y1=np.array([872, 849, 851, 899])

##Grain 56A
x0=np.array([627, 657, 646])
y0=np.array([516, 577, 537])
x1=np.array([663, 666, 662])
y1=np.array([484, 569, 522])

##Grain 56B
x0=np.array([680, 648, 634,687,713,698])
y0=np.array([554, 521, 490,501,568,569])
x1=np.array([709, 668, 654,699,733,722])
y1=np.array([502, 489, 454,482,535,529])

##Grain 96
x0=np.array([2, 15, 50, 2])
y0=np.array([920, 1004, 1005, 978])
x1=np.array([8, 47, 68, 29])
y1=np.array([914, 961, 981, 943])

## Grain 18
x0=np.array([420, 415, 432, 427])
y0=np.array([87, 80, 97, 95])
x1=np.array([441, 436, 449, 450])
y1=np.array([67, 60, 81, 73])

##Grain 101
x0=np.array([830, 874, 904, 953, 984])
y0=np.array([1007, 997, 1000, 1015, 981])
x1=np.array([898, 957, 962, 1027, 1038])
y1=np.array([1088, 1095, 1070, 1102, 1045])

## Grain 40
x0=np.array([716, 759,732,791,811])
y0=np.array([318, 300,362,295,315])
x1=np.array([678, 744,703,724,786])
y1=np.array([405, 332,432,449,373])

##Grain 0
x0=np.array([57, 24, 7])
y0=np.array([36, 35, 28])
x1=np.array([70, 37, 19])
y1=np.array([3, 3, 3])

##Grain 4
x0=np.array([572, 568])
y0=np.array([57, 38])
x1=np.array([521, 528])
y1=np.array([17, 9])

##Grain 9
x0=np.array([868, 882, 894, 888])
y0=np.array([77, 74, 69, 41])
x1=np.array([850, 852, 856, 868])
y1=np.array([63, 50, 40, 25])

##Grain 15
x0=np.array([749, 729, 720, 775])
y0=np.array([85, 75, 66, 86])
x1=np.array([766, 739, 728, 789])
y1=np.array([40, 44, 43, 52])

##Grain 19
x0=np.array([599, 629, 549])
y0=np.array([194, 169, 120])
x1=np.array([535, 563, 615])
y1=np.array([131, 104, 185])

##Grain 23b   121
x0=np.array([958])
y0=np.array([102])
x1=np.array([982])
y1=np.array([142])

##Grain 23   57
x0=np.array([1025, 939, 990])
y0=np.array([189, 126, 144])
x1=np.array([1036, 950, 1006])
y1=np.array([168, 103, 117])

##Grain 22
x0=np.array([659, 741, 621])
y0=np.array([301, 131, 279])
x1=np.array([740, 638, 675])
y1=np.array([179, 286, 197])

##Grain 36
x0=np.array([101, 51, 81])
y0=np.array([304, 334, 292])
x1=np.array([182, 156, 53])
y1=np.array([349, 394, 276])

## Grain 43
x0=np.array([300, 279])
y0=np.array([391, 387])
x1=np.array([325, 288])
y1=np.array([347, 366])

## Grain 59
x0=np.array([982, 962])
y0=np.array([531, 556])
x1=np.array([1085, 1064])
y1=np.array([660, 682])

## Grain 104
x0=np.array([768, 710])
y0=np.array([1030, 1131])
x1=np.array([708, 776])
y1=np.array([1106, 1048])

## Grain 105
x0=np.array([1152, 1177])
y0=np.array([1141, 1147])
x1=np.array([1211, 1212])
y1=np.array([1060, 1096])

## Grain 81
x0=np.array([1039, 1042])
y0=np.array([859, 844])
x1=np.array([996, 1014])
y1=np.array([800, 803])

## Grain 66
x0=np.array([346, 330, 362, 321])
y0=np.array([590, 615, 577, 651])
x1=np.array([382, 384, 422, 353])
y1=np.array([625, 666, 634, 682])

## Grain 73
x0=np.array([266, 290])
y0=np.array([715, 726])
x1=np.array([302, 319])
y1=np.array([663, 687])

## Grain 10
x0=np.array([1046, 1016, 1097])
y0=np.array([157, 100, 173])
x1=np.array([1138, 1093, 1145])
y1=np.array([45, 7, 117])

## Grain 57       58.7
x0=np.array([62, 22])
y0=np.array([574, 539])
x1=np.array([104, 60])
y1=np.array([504, 475])

## Grain 57       118
x0=np.array([131])
y0=np.array([508])
x1=np.array([187])
y1=np.array([613])

## Grain 68
x0=np.array([997, 991])
y0=np.array([712, 683])
x1=np.array([948, 973])
y1=np.array([634, 647])

## Grain 1
x0=np.array([261])
y0=np.array([184])
x1=np.array([127])
y1=np.array([72])

## Grain 65
x0=np.array([944, 933])
y0=np.array([598, 600])
x1=np.array([914, 909])
y1=np.array([569, 575])

## Grain 74
x0=np.array([755, 763, 771, 848])
y0=np.array([725, 709, 698, 725])
x1=np.array([802, 828, 834, 788])
y1=np.array([769, 767, 755, 669])

##Grain 56B
x0=np.array([680, 648, 634,687,713,698])
y0=np.array([554, 521, 490,501,568,569])
x1=np.array([709, 668, 654,699,733,722])
y1=np.array([502, 489, 454,482,535,529])

##Grain 56A
x0=np.array([627, 657, 646, 644, 680])
y0=np.array([516, 577, 537, 555, 473])
x1=np.array([663, 666, 662, 656, 687])
y1=np.array([483, 569, 522, 544, 466])

## Grain 75
x0=np.array([44, 27, 83])
y0=np.array([796, 898, 968])
x1=np.array([155, 204, 232])
y1=np.array([689, 722, 824])

## Grain 78
x0=np.array([363])
y0=np.array([803])
x1=np.array([423])
y1=np.array([757])

## Grain 88
x0=np.array([891, 897])
y0=np.array([887, 904])
x1=np.array([912, 932])
y1=np.array([863, 869])

## Grain 89
x0=np.array([623, 674])
y0=np.array([905, 856])
x1=np.array([681, 633])
y1=np.array([871, 880])

##Grain 36
x0=np.array([101, 51, 81])
y0=np.array([304, 334, 292])
x1=np.array([182, 156, 53])
y1=np.array([349, 394, 276])

##Grain 36b
x0=np.array([99, 83, 83])
y0=np.array([293, 302, 391])
x1=np.array([161, 49, 152])
y1=np.array([254, 325, 348])

## Grain 94
x0=np.array([1051])
y0=np.array([875])
x1=np.array([1077])
y1=np.array([907])

## Grain 100
x0=np.array([582, 624])
y0=np.array([978, 965])
x1=np.array([644, 658])
y1=np.array([1024, 991])

## Grain 105
x0=np.array([1159, 1178])
y0=np.array([1137, 1146])
x1=np.array([1209, 1211])
y1=np.array([1062, 1099])

## Grain 16
x0=np.array([514, 527])
y0=np.array([131, 115])
x1=np.array([489, 506])
y1=np.array([109, 97])

## Grain 16
x0=np.array([69, 61, 78])
y0=np.array([189, 176, 197])
x1=np.array([8, 8, 8])
y1=np.array([252, 230, 270])

## Grain 25
x0=np.array([413, 449])
y0=np.array([292, 270])
x1=np.array([413, 451])
y1=np.array([406, 335])

## Grain 32
x0=np.array([1214, 1212])
y0=np.array([235, 270])
x1=np.array([1201, 1197])
y1=np.array([224, 256])

## Grain 34
x0=np.array([1174, 1169])
y0=np.array([244, 259])
x1=np.array([1143, 1135])
y1=np.array([213, 228])

## Grain 39
x0=np.array([1159])
y0=np.array([324])
x1=np.array([1203])
y1=np.array([376])

## Grain 40
x0=np.array([733, 717])
y0=np.array([343, 384])
x1=np.array([725, 707])
y1=np.array([321, 355])

## Grain 48    68.5
x0=np.array([303, 337])
y0=np.array([494, 487])
x1=np.array([335, 363])
y1=np.array([409, 418])

## Grain 48    120.4
x0=np.array([324, 294])
y0=np.array([552, 498])
x1=np.array([303, 281])
y1=np.array([515, 476])

## Grain 54
x0=np.array([1193, 1196])
y0=np.array([511, 526])
x1=np.array([1211, 1213])
y1=np.array([463, 468])

## Grain 60
x0=np.array([412])
y0=np.array([555])
x1=np.array([399])
y1=np.array([542])

## Grain 67
x0=np.array([753, 739])
y0=np.array([696, 709])
x1=np.array([704, 663])
y1=np.array([631, 606])

## Grain 69
x0=np.array([8, 4, 14])
y0=np.array([757, 694, 717])
x1=np.array([82, 22, 51])
y1=np.array([677, 674, 674])

## Grain 70
x0=np.array([234, 225])
y0=np.array([672, 646])
x1=np.array([207, 213])
y1=np.array([638, 631])

## Grain 72
x0=np.array([1095])
y0=np.array([738])
x1=np.array([1120])
y1=np.array([698])

## Grain 78b   152
x0=np.array([298])
y0=np.array([760])
x1=np.array([347])
y1=np.array([782])

## Grain 79
x0=np.array([855])
y0=np.array([765])
x1=np.array([886])
y1=np.array([713])

## Grain 80
x0=np.array([1192, 1204, 1206])
y0=np.array([936, 923, 888])
x1=np.array([1152, 1151, 1156])
y1=np.array([891, 862, 831])

## Grain 82
x0=np.array([675, 553])
y0=np.array([809, 868])
x1=np.array([612, 527])
y1=np.array([766, 852])

## Grain 83
x0=np.array([1059, 1063])
y0=np.array([849, 797])
x1=np.array([1085, 1092])
y1=np.array([818, 762])

## Grain 85
x0=np.array([773, 739])
y0=np.array([829, 856])
x1=np.array([747, 691])
y1=np.array([809, 820])

## Grain 91
x0=np.array([415, 368, 275])
y0=np.array([1109, 1095, 1026])
x1=np.array([458, 421, 316])
y1=np.array([1042, 1013, 965])

## Grain 92
x0=np.array([566, 603, 549])
y0=np.array([963, 947, 907])
x1=np.array([524, 571, 498])
y1=np.array([919, 923, 870])

## Grain 99
x0=np.array([225, 184])
y0=np.array([1074, 1050])
x1=np.array([259, 219])
y1=np.array([1019, 996])

## Grain 106
x0=np.array([256, 447])
y0=np.array([1149, 1149])
x1=np.array([238, 436])
y1=np.array([1121, 1130])

## Grain 106b
x0=np.array([188, 306])
y0=np.array([1148, 1147])
x1=np.array([214, 321])
y1=np.array([1094, 1118])

## Grain 107
x0=np.array([64, 130])
y0=np.array([1148, 1148])
x1=np.array([81, 138])
y1=np.array([1097, 1130])

## Grain 109
x0=np.array([835, 897])
y0=np.array([1147, 1148])
x1=np.array([890, 944])
y1=np.array([1106, 1111])

## Grain 114
x0=np.array([549])
y0=np.array([1149])
x1=np.array([521])
y1=np.array([1128])

##### MAP2    

## Grain 47
#x0=np.array([1087, 1211, 1133, 1134])
#y0=np.array([323, 434, 464, 371])
#x1=np.array([1123, 1179, 1109, 1103])
#y1=np.array([379, 385, 425, 325])

## Grain 8
#x0=np.array([728, 715, 815])
#y0=np.array([124, 73, 87])
#x1=np.array([792, 806, 764])
#y1=np.array([163, 126, 58])

## Grain 11
#x0=np.array([557, 573])
#y0=np.array([76, 78])
#x1=np.array([591, 606])
#y1=np.array([30, 36])

## Grain 12
x0=np.array([280, 225, 284])
y0=np.array([43, 210, 64])
x1=np.array([349, 282, 342])
y1=np.array([105, 261, 115])

## Grain 13
x0=np.array([466, 454, 444])
y0=np.array([109, 119, 135])
x1=np.array([433, 428, 420])
y1=np.array([74, 92, 110])

## Grain 21
x0=np.array([49, 56])
y0=np.array([172, 205])
x1=np.array([22, 36])
y1=np.array([163, 196])

## Grain 22
x0=np.array([1108, 1112, 1122])
y0=np.array([187, 195, 189])
x1=np.array([1131, 1134, 1136])
y1=np.array([156, 168, 172])

## Grain 24
x0=np.array([488, 519])
y0=np.array([221, 261])
x1=np.array([493, 523])
y1=np.array([203, 233])

## Grain 25
x0=np.array([1028, 1056, 1034])
y0=np.array([169, 182, 151])
x1=np.array([1048, 1032, 1058])
y1=np.array([188, 157, 174])

## Grain 27
x0=np.array([1196, 1187])
y0=np.array([227, 188])
x1=np.array([1159, 1165])
y1=np.array([203, 174])

## Grain 30
x0=np.array([476, 461, 432])
y0=np.array([230, 236, 215])
x1=np.array([449, 442, 420])
y1=np.array([211, 222, 204])

## Grain 36   35.42
x0=np.array([56])
y0=np.array([380])
x1=np.array([121])
y1=np.array([334])

## Grain 36   152.84
x0=np.array([44, 50])
y0=np.array([290, 311])
x1=np.array([104, 111])
y1=np.array([318, 341])

## Grain 39
x0=np.array([731, 751, 791, 707])
y0=np.array([319, 286, 329, 306])
x1=np.array([756, 766, 810, 718])
y1=np.array([287, 268, 296, 289])

## Grain 41
x0=np.array([394, 413, 400, 406])
y0=np.array([312, 388, 307, 303])
x1=np.array([442, 369, 440, 443])
y1=np.array([379, 328, 363, 354])

## Grain 42
x0=np.array([608, 677])
y0=np.array([356, 285])
x1=np.array([679, 691])
y1=np.array([275, 270])

## Grain 45
x0=np.array([209, 264])
y0=np.array([295, 299])
x1=np.array([234, 282])
y1=np.array([339, 330])

## Grain 45
x0=np.array([209, 264])
y0=np.array([295, 299])
x1=np.array([234, 282])
y1=np.array([339, 330])

## grain 53
x0=np.array([839, 861, 816, 850, 789, 811])
y0=np.array([518, 554, 512, 521, 441, 508])
x1=np.array([874, 925, 860, 885, 814, 837])
y1=np.array([471, 467, 456, 477, 407, 476])

## Grain 2
#x0=np.array([161,194, 204, 142])
#y0=np.array([10,29, 33, 212])
#x1=np.array([70,137, 172, 177])
#y1=np.array([158,121, 87, 155])

## Grain 75
x0=np.array([781, 745, 762])
y0=np.array([792, 780, 789])
x1=np.array([748, 720, 731])
y1=np.array([724, 727, 726])

## Grain 76
#x0=np.array([964, 954, 948, 984, 992])
#y0=np.array([834, 811, 753, 839, 867])
#x1=np.array([1022, 1009, 920, 1008, 1028])
#y1=np.array([766, 747, 786, 810, 826])

## Grain 98
#x0=np.array([532])
#y0=np.array([1023])
#x1=np.array([573])
#y1=np.array([949])

## Grain 98b
x0=np.array([503, 487, 499])
y0=np.array([966, 995, 971])
x1=np.array([508, 497, 511])
y1=np.array([977, 1017, 1002])

## Grain 110
#x0=np.array([714, 755, 742, 694])
#y0=np.array([1147, 1148, 1145, 1102])
#x1=np.array([749, 790, 777, 713])
#y1=np.array([1098, 1101, 1096, 1078])

## Grain 49
#x0=np.array([6])
#y0=np.array([348])
#x1=np.array([11])
#y1=np.array([328])

## Grain 54
x0=np.array([634])
y0=np.array([517])
x1=np.array([679])
y1=np.array([471])

## Grain 55
x0=np.array([773, 731, 921])
y0=np.array([527, 532, 581])
x1=np.array([732, 683, 888])
y1=np.array([577, 591, 626])

## Grain 56
x0=np.array([5, 14, 17, 20])
y0=np.array([408, 444, 431, 440])
x1=np.array([21, 3, 3, 2])
y1=np.array([427, 431, 415, 424])

## Grain 57
x0=np.array([68, 32, 85, 85])
y0=np.array([469, 571, 551, 538])
x1=np.array([81, 44, 98, 100])
y1=np.array([445, 544, 527, 510])

## Grain 58
x0=np.array([1139, 1146])
y0=np.array([495, 474])
x1=np.array([1170, 1167])
y1=np.array([550, 512])

## Grain 59
x0=np.array([371, 434])
y0=np.array([516, 535])
x1=np.array([405, 377])
y1=np.array([530, 512])

## Grain 60
x0=np.array([1125])
y0=np.array([661])
x1=np.array([1093])
y1=np.array([640])

## Grain 61
x0=np.array([531, 493])
y0=np.array([564, 605])
x1=np.array([509, 474])
y1=np.array([537, 583])

## Grain 62
x0=np.array([596, 550])
y0=np.array([679, 686])
x1=np.array([565, 534])
y1=np.array([642, 668])

## Grain 70
x0=np.array([658, 684])
y0=np.array([691, 702])
x1=np.array([628, 632])
y1=np.array([656, 642])

## Grain 72
x0=np.array([932, 941, 930])
y0=np.array([652, 650, 641])
x1=np.array([949, 956, 959])
y1=np.array([639, 637, 618])

## Grain 75b
x0=np.array([844, 787, 772, 786])
y0=np.array([880, 913, 912, 893])
x1=np.array([859, 803, 786, 795])
y1=np.array([843, 878, 877, 875])

## Grain 84
#x0=np.array([114, 165, 100, 131])
#y0=np.array([953, 908, 884, 934])
#x1=np.array([74, 144, 84, 81])
#y1=np.array([911, 889, 869, 887])

## Grain 78
x0=np.array([283])
y0=np.array([808])
x1=np.array([352])
y1=np.array([727])

## Grain 81
x0=np.array([1119])
y0=np.array([905])
x1=np.array([1085])
y1=np.array([860])

## Grain 93
x0=np.array([321, 315])
y0=np.array([1082, 1072])
x1=np.array([346, 347])
y1=np.array([1028, 1004])

## Grain 95
x0=np.array([986])
y0=np.array([959])
x1=np.array([968])
y1=np.array([940])

## Grain 99    130
x0=np.array([770])
y0=np.array([1038])
x1=np.array([721])
y1=np.array([975])

## Grain 99    130
x0=np.array([803])
y0=np.array([1071])
x1=np.array([820])
y1=np.array([1024])

## Grain 103
x0=np.array([984, 967])
y0=np.array([1118, 1098])
x1=np.array([945, 946])
y1=np.array([1088, 1082])

## Grain 96
x0=np.array([158])
y0=np.array([949])
x1=np.array([165])
y1=np.array([987])

## Grain 104
x0=np.array([28])
y0=np.array([1060])
x1=np.array([42])
y1=np.array([1085])

## Grain 105
x0=np.array([1107])
y0=np.array([1053])
x1=np.array([1116])
y1=np.array([1067])

## Grain 105
x0=np.array([591, 597])
y0=np.array([1112, 1124])
x1=np.array([623, 616])
y1=np.array([1090, 1109])

## Grain 105
x0=np.array([276])
y0=np.array([1114])
x1=np.array([252])
y1=np.array([1145])

## Grain 48
x0=np.array([1027])
y0=np.array([429])
x1=np.array([997])
y1=np.array([382])

## Grain 39
x0=np.array([731, 751, 791, 707])
y0=np.array([319, 286, 329, 306])
x1=np.array([756, 766, 810, 718])
y1=np.array([287, 268, 296, 289])

## Grain 56
#x0=np.array([5, 14, 17, 20])
#y0=np.array([408, 444, 431, 440])
#x1=np.array([21, 3, 3, 2])
#y1=np.array([427, 431, 415, 424])

## Grain 20 IRR
x0=np.array([333,316,309,345,352,314,322])
y0=np.array([83,91,105,77,72,126,147])
x1=np.array([428,419,402,431,433,390,367])
y1=np.array([167,182,188,153,143,192,185])

runRDR(x0,y0,x1,y1,length=2.5, cond=10,rcond=0,SelDicMap=DicMap)

### Locate a grain of interest

In [ ]:
SelDicMap=DicMap

SelDicMap.locateGrainID(displaySelected=True, vmax=10, dilateBoundaries=True)
maxStrain = np.max(SelDicMap.max_shear)
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain))

In [ ]:
Grain=DicMap.grainList[69]
ebsdGrain = Grain.ebsdGrain
print("Euler angles: ")
print("",ebsdGrain.refOri.eulerAngles()*180/np.pi, "\n")

In [ ]:
SelDicMap.grainList[SelDicMap.currGrainId].plotMaxShear(vmax=10,plotSlipTraces=True)

In [ ]:
SelDicMap.grainList[SelDicMap.currGrainId].ebsdGrain.plotMisOri(vmax=6)

In [ ]:
SelDicMap.grainList[SelDicMap.currGrainId].ebsdGrain.buildMisOriList(parallel=True,calcAxis=True)
SelDicMap.grainList[SelDicMap.currGrainId].ebsdGrain.plotMisOri(component=3,vmin=-3,vmax=3)

In [ ]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
EbsdMap.grainList[ebsdGrainId].plotMisOri()

In [ ]:
plt.hist([142.17,141.25,139.54,66.6,133.47,46.65,137.14,69.6,49.3,41.11,47.38,121.48,57.29,138.75,133.04,69.4,34.63,138.03,140.71,93,68.52,35.61, 59.46,65.63,128.387,58.7,57.46,67.82,121.68,133.83,47.09,121.52,124.29,137,132.72,47.19,136.29,56.71,37.04,44.09,135.32,52.96,55.96,129.47,30.22,45.67,48.6,141.42,50.45,143.11,125.84,130.39,56.72,53.5,129.07,143.87,58.41,38.02,72.93,119.45,55.33,49.53,49.82,131.17,141.66,143.04],bins=18,range=[0,180])

### Grain info

In [ ]:
from builtins import print
Grain=DicMap.grainList[DicMap.currGrainId]
ebsdGrain = Grain.ebsdGrain

print("ID: {}".format(DicMap.currGrainId))
print("Area: {} px^2".format(len(ebsdGrain)*0.25))
print("Width in DIC frame (px): {0}".format(Grain.extremeCoords[2]-Grain.extremeCoords[0]))
print("Height in DIC frame (px): {0}".format(Grain.extremeCoords[3]-Grain.extremeCoords[1]))
minStrain = np.min(Grain.maxShearList)*100
meanStrain = np.mean(Grain.maxShearList)*100
maxStrain = np.max(Grain.maxShearList)*100
print("Min effective shear strain: {0:.3f} %".format(minStrain))
print("Mean effective shear strain: {0:.3f} %".format(meanStrain))
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain))

ebsdGrain.calcSlipTraces()
print("Euler angles: ")
print("",ebsdGrain.refOri.eulerAngles()*180/np.pi, "\n")

angle=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi

if ebsdGrain.refOri.eulerAngles()[1]*180/np.pi > 90:
    angle=180-angle
    
schdFacs = ebsdGrain.averageSchmidFactors
slipTraces = ebsdGrain.slipTraces
for ssGroup, sfGroup, slipTrace in zip(EbsdMap.slipSystems, schdFacs, slipTraces):
    stAngle = np.arctan(slipTrace[1]/slipTrace[0]) *180/np.pi
    if stAngle < 0: stAngle += 180
    if stAngle > 180: stAngle -= 180
    print("Slip Plane Normal {0:s}\tSlip trace: ({1:.3f}, {2:.3f})\tAngle: {3:.1f}".
          format(ssGroup[0].slipPlaneLabel, slipTrace[0], slipTrace[1], stAngle))
    for ss, sf in zip(ssGroup, sfGroup):
        slipDirSample=ebsdGrain.refOri.conjugate.transformVector(ss.slipDir)
        print("\tSlip Direction {0:s}\tSchmid factor: {1:.3f}\tRDR: {2:.3f}".
              format(ss.slipDirLabel, sf,-slipDirSample[0]/slipDirSample[1]))
        

### Grain Stats

In [ ]:
from builtins import print
print("GrainID\tArea\t<ESS>\tEuler1\tEuler2\tEuler3\tAngC\tMaxESS\tMinESS\tBaSc\tPrSc\tPySc")
print("\t(px^2)\t(%)\t(deg)\t(deg)\t(deg)\t(deg)\t(%)\t(%)")
meanStrains=[]
angC=[]
maxStrains=[]
minStrains=[]
errors=[0,0]
baschmidlist=[]
prschmidlist=[]
pyschmidlist=[]

SelDicMap=DicMap2

for x in range(0,len(SelDicMap.grainList)):
     
    ebsdGrain = SelDicMap[x].ebsdGrain
    meanStrain = np.mean(SelDicMap.grainList[x].maxShearList)*100
    maxStrain = np.max(SelDicMap.grainList[x].maxShearList)*100
    minStrain = np.min(SelDicMap.grainList[x].maxShearList)*100
    ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
    ebsdGrain.calcSlipTraces()
    Euler1=ebsdGrain.refOri.eulerAngles()[0]*180/np.pi
    Euler2=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi
    Euler3=ebsdGrain.refOri.eulerAngles()[2]*180/np.pi
    angle=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi
    meanStrains.append(meanStrain)
    maxStrains.append(maxStrain)
    minStrains.append(minStrain)
    if angle > 90:
        angle=180-angle
    angC.append(angle)
    baschmid=np.max(ebsdGrain.averageSchmidFactors[0])
    prschmid=np.max((ebsdGrain.averageSchmidFactors[1],ebsdGrain.averageSchmidFactors[2],ebsdGrain.averageSchmidFactors[3]))
    pyschmid=np.max((ebsdGrain.averageSchmidFactors[4],ebsdGrain.averageSchmidFactors[5],ebsdGrain.averageSchmidFactors[6],ebsdGrain.averageSchmidFactors[7],ebsdGrain.averageSchmidFactors[8]))
    
    baschmidlist.append(baschmid)
    prschmidlist.append(prschmid)
    pyschmidlist.append(pyschmid)
    
    print("{0}\t{1:.2f}\t{2:.2f}\t{3:.2f}\t{4:.2f}\t{5:.2f}\t{6:.2f}\t{7:.2f}\t{8:.2f}\t{9:.3f}\t{10:.3f}\t{11:.3f}".format(x,len(ebsdGrain)*0.25,meanStrain,Euler1,Euler2,Euler3,angle,maxStrain,minStrain,baschmid,prschmid,pyschmid))
  
lowe=[i - j for i, j in zip(meanStrains, minStrains)]
highe=[k - l for k, l in zip(maxStrains, meanStrains)]
errors[0]=lowe
errors[1]=highe
#fig=plt.errorbar(angC,meanStrains,yerr=errors, marker='x',linestyle="None",capsize=5)
#plt.scatter(angC,[k / l for k, l in zip(maxStrains, meanStrains)])
fig,(ax0,ax1,ax2) = plt.subplots(1, 3, figsize=(4,3))
ax0.hist(baschmidlist,range=[0,0.5], bins=10)
ax0.title.set_text('Ba')
ax1.hist(prschmidlist,range=[0,0.5], bins=10)
ax1.title.set_text('Pr')
ax2.hist(pyschmidlist,range=[0,0.5], bins=10)
ax2.title.set_text('Py')

fig2,(ax0) = plt.subplots(1, 1, figsize=(4,3))
ax0.hist(meanStrains,range=[0,10],bins=20)

### Grain profile

In [ ]:
def plot_profile(def_map,start_point,end_point):
    x0,y0  = start_point[0], start_point[1] # These are in _pixel_ coordinates!!
    x1,y1 = end_point[0], end_point[1]
    profile_length=np.sqrt((y1-y0)**2+(x1-x0)**2)
    num = np.round(profile_length)
    x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)

    # Extract the values along the line, using cubic interpolation
    zi = scipy.ndimage.map_coordinates(np.transpose(def_map), np.vstack((x,y)))
    return zi

In [ ]:
print(corr_profile2)

In [ ]:
print(grainMisOri[63])

In [ ]:
component=3
component=int(component)
DicMap2.currGrainId=75
Grain=DicMap2.grainList[DicMap2.currGrainId]
ebsdGrain=Grain.ebsdGrain
#ebsdGrain=EbsdMap.grainList[30]

ebsdGrain.buildMisOriList(calcAxis=True)
                       
f, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(16,8))

##### EBSD DATA

ex0, ey0, exmax, eymax = ebsdGrain.extremeCoords
grainMisOri = np.full((eymax - ey0 + 1, exmax - ex0 + 1), 0, dtype=float)


if component in [4]:
    plotData = np.arccos(ebsdGrain.misOriList) * 360 / np.pi

else:    
    plotData = np.array(ebsdGrain.misOriAxisList)[:, component - 1] * 180 / np.pi

for coord, misOri in zip(ebsdGrain.coordList, plotData):
    grainMisOri[coord[1] - ey0, coord[0] - ex0] = misOri

##### HR-DIC DATA

unzippedCoordlist = list(zip(*Grain.coordList))
x0, y0, xmax, ymax = Grain.extremeCoords
grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), 0, dtype=float)

for coord, maxShear in zip(Grain.coordList, Grain.maxShearList):
    grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear

##### PLOTTING
    
ax1.imshow(grainMaxShear,vmin=0,vmax=0.2)

if component in [4]:
    ax0.imshow(grainMisOri, interpolation='none', vmin=0, vmax=6, cmap='viridis')
else:
    ax0.imshow(grainMisOri, interpolation='none', vmin=-6, vmax=6, cmap='coolwarm')

point1=[150,15]
point2=[240,70]

point1=[110,125]
point2=[190,60]  ##28DIC

point1=[50,8]
point2=[200,130] 

point1=[25,115]    ##75DIC
point2=[215,15]

point1=[30,170]    ##75DIC
point2=[210,210]

ax1.scatter(point1[0],point1[1])
ax1.scatter(point2[0],point2[1])

yratio=(ymax-y0)/(eymax-ey0)
xratio=(xmax-x0)/(exmax-ex0)

ebsdpoint1=[point1[0]/xratio,point1[1]/yratio]
ebsdpoint2=[point2[0]/xratio,point2[1]/yratio]

ebsdpoint1=[point1[0]/xratio,point1[1]/yratio]
ebsdpoint2=[point2[0]/xratio,point2[1]/yratio]
#ebsdpoint1=[20,39]
#ebsdpoint2=[75,82]

print(ebsdpoint1,ebsdpoint2)

ax0.scatter(ebsdpoint1[0],ebsdpoint1[1])
ax0.scatter(ebsdpoint2[0],ebsdpoint2[1])

##### PROFILES
corr_profile=plot_profile(grainMaxShear,point1,point2)
#corr_profile=skimage.measure.profile_line(grainMaxShear,point1,point2, order=1)
corr_profile=np.array(corr_profile)
ax3.plot(range(len(corr_profile)),corr_profile,'-.')

corr_profile2=plot_profile(grainMisOri,ebsdpoint1,ebsdpoint2)
#corr_profile2=skimage.measure.profile_line(grainMisOri,ebsdpoint1,ebsdpoint2,order=1)
ax2.plot(range(len(corr_profile2)),corr_profile2,'-.')

f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile_dic.txt', 'wb')
np.savetxt(f,corr_profile,fmt='%.6f')
f.close()

f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile_ebsd.txt', 'wb')
np.savetxt(f,corr_profile2,fmt='%.6f')
f.close()

In [ ]:
import skimage
corr_profile2=plot_profile(grainMisOri,ebsdpoint1,ebsdpoint2)
print(corr_profile2)
corr_profile2=skimage.measure.profile_line(grainMisOri,ebsdpoint1,ebsdpoint2)
print(corr_profile2)

In [ ]:
Grain=DicMap.grainList[DicMap.currGrainId]
unzippedCoordlist = list(zip(*Grain.coordList))
x0 = min(unzippedCoordlist[0])
y0 = min(unzippedCoordlist[1])
xmax = max(unzippedCoordlist[0])
ymax = max(unzippedCoordlist[1])
grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), 0, dtype=float)

for coord, maxShear in zip(Grain.coordList, Grain.maxShearList):
    grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear

print(grainMaxShear)
    
corr_profile2=plot_profile(grainMaxShear,[90,90],[260,260])

#f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
np.savetxt('corr_profile2.txt',corr_profile2,fmt='%.6f')
#f.close()

In [ ]:
newvalues = [x for x in DicMap2.grainList[DicMap2.currGrainId].maxShearList if x != 0]

hist, bins = np.histogram(newvalues, bins=800,range=[0,0.6],density=True)
#hist=np.log10(hist*100)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
np.savetxt('grainnirr.csv', (hist), delimiter=',')

### Grain info

In [ ]:
#Calculate FWHM given peak positions (indexes) and array (x,y) 
def FWHM(indexes, Xorig,Yori):
    fwhmList=[]   #Define empty array for resulting FWHMs
    indexes+=20
    Xorig=np.arange(220)
    Yorig=np.concatenate((Yori[160:180],Yori,Yori[0:20]))
    
    for indexes in indexes:
        X=Xorig[indexes-18:indexes+18]
        Y=Yorig[indexes-18:indexes+18]
        half_max = max(Y) / 2.
        #find when function crosses line half_max (when sign of diff flips)
        #take the 'derivative' of signum(half_max - Y[])
        d = np.sign(half_max - np.array(Y[0:-1])) - np.sign(half_max - np.array(Y[1:]))
        #plot(X,d) #if you are interested
        #find the left and right most indexes
        left_idx = X[np.where( d > 0.1 )]
        right_idx = X[np.where( d < -0.1 )]
        FWHM=0
        if len(right_idx)==1 and len(left_idx)==1:
            FWHM=right_idx - left_idx #return the difference (full width)
        fwhmList.append(np.int(FWHM))
    fwhmList=np.asarray(fwhmList)
    return fwhmList

In [ ]:
SelDicMap=DicMap
#52 DICMAP2
#75 DICMAPIRR
for grainID in range (10,11):
    ebsdGrain = SelDicMap[grainID].ebsdGrain
    minStrain = np.min(SelDicMap.grainList[grainID].maxShearList)*100
    meanStrain = np.mean(SelDicMap.grainList[grainID].maxShearList)*100
    maxStrain = np.max(SelDicMap.grainList[grainID].maxShearList)*100
    medianStrain = np.median(SelDicMap.grainList[grainID].maxShearList)*100
    ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
    ebsdGrain.calcSlipTraces()
    np.set_printoptions(precision=3)
##### GENERATE MAX SHEARS
    x0, y0, xmax, ymax = SelDicMap[grainID].extremeCoords
    grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)
    for coord, maxShear in zip(SelDicMap.grainList[grainID].coordList, SelDicMap.grainList[grainID].maxShearList):
        grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear
##### WRITE TEXT  
        B = []
    def print(str):
        global B
        B.append(str)
    #from builtins import print
    print("Grain ID: {0:.0f}".format(grainID))
    print("Area: {} um^2".format(len(SelDicMap.grainList[grainID].coordList)*0.006103515625))
    print("Min effective shear strain: {0:.3f} %".format(minStrain))
    print("Mean effective shear strain: {0:.3f} %".format(meanStrain))
    print("Max effective shear strain: {0:.3f} %".format(maxStrain))
    print("Median effective shear strain: {0:.3f} %\n".format(medianStrain))
    print(ebsdGrain.refOri.eulerAngles()*180/np.pi)
    angle=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi
    if ebsdGrain.refOri.eulerAngles()[1]*180/np.pi > 90:
        angle=180-angle
    print("Angle from c-axis?: {0:.1f} degrees\n".format(angle))
##### SHOW MAX SHEAR
    fig, ((ax0, ax1, ax2, ax3), (ax4, ax5, ax6, ax7)) = plt.subplots(2, 4, figsize=(20, 10))
    im=ax1.imshow(grainMaxShear,cmap='viridis',vmin=0,vmax=0.1)
    fig.colorbar(im, ax=ax1,label="Effective shear strain (%)")
    scalebar = ScaleBar(pixelinm) # 1 pixel = 0.2 meter
    ax1.add_artist(scalebar)
    ax1.axis('off')
##### CALCULATE RADON
    grainMaxShear[np.isnan(grainMaxShear)]=0
    sin_map = tf.radon(grainMaxShear,circle=False,theta=list(range(90,180)) + list(range(90)))
    profile = np.max(sin_map, axis=0)
    x = np.arange(180)
    y = profile
    base = peakutils.baseline(y, 2)
    y=y-base
    indexes = peakutils.indexes(y, thres=0.2, min_dist=20)
    peaks = peakutils.interpolate(x, y, ind=indexes)
    shearBandAngles = peaks
    breadths=FWHM(indexes,x,y)
    for ID in range(0,len(shearBandAngles)):
        if shearBandAngles[ID] > 180:
            shearBandAngles[ID]-=180
    print("Angles detected: {}".format(shearBandAngles))
    print("FWHMs: {}".format(breadths))
###### CALCULATE SLIP TRACE ANGLES
    schdFacs = ebsdGrain.averageSchmidFactors
    slipTraces = ebsdGrain.slipTraces
    for ssGroup, sfGroup, slipTrace in zip(EbsdMap.slipSystems, schdFacs, slipTraces):
        stAngle = np.arctan(slipTrace[1]/slipTrace[0]) *180/np.pi
        stAngle += 2
        if stAngle < 0: stAngle += 180
        if stAngle > 180: stAngle -= 180
        print("{0:s}     Angle: {1:.1f}".
              format(ssGroup[0].slipPlaneLabel, stAngle))
        for ss, sf in zip(ssGroup, sfGroup):
            print("{0:s}     {1:.3f}".format(ss.slipDirLabel, sf))

    colours = ["red", "blue", "blue", "blue", "green", "green", "green", "green", "green", "green"]
    xPos = int((xmax - x0) / 2)
    yPos = int((ymax - y0) / 2)
##### GENERATE FIGURES
    ax2.axis('off')
    ax3.axis('off')
    ax0.imshow(sin_map)
    ax4.plot(x,y)
    for xc in peaks:
        ax4.axvline(x=xc)
##### PLOTTING RADON AND FWHM
    plotRadon='False'
    if plotRadon=='True':
        shearBandAngles = -(180-shearBandAngles) * np.pi / 180
        breadths = -breadths * np.pi / 180
        shearBandVectors = np.array((np.cos(shearBandAngles), np.sin(shearBandAngles)))
        ax3.quiver(xPos, yPos, shearBandVectors[0], shearBandVectors[1], scale=0.9, pivot="middle",
                           color='orange', headwidth=1, headlength=0)
        fwhmAngles = []
        for breadth, angle in zip(breadths, shearBandAngles):
            fwhmAngles.append(angle+(breadth/2))
            fwhmAngles.append(angle-(breadth/2))
        fwhmVectors = np.array((np.cos(fwhmAngles), np.sin(fwhmAngles)))
        ax3.quiver(xPos, yPos, fwhmVectors[0], fwhmVectors[1], scale=0.9, pivot="middle",
                           color='orange', headwidth=1, headlength=0)
###### PLOT SLIP TRACE ANGLES
    for i, slipTrace in enumerate(slipTraces):
        colour = colours[len(colours) - 1] if i >= len(colours) else colours[i]
        ax3.quiver(xPos, yPos, slipTrace[0], slipTrace[1], scale=1, pivot="middle",
                              color=colour, headwidth=1, headlength=0)
###### MAKE HISTOGRAM
    newvalues = [x for x in SelDicMap.grainList[grainID].maxShearList if x != 0]
    hist, bins = np.histogram(newvalues, bins=200,range=[0,0.6],density=True)
    #hist=np.log10(hist*100)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    ax7.plot(center, hist, marker='x',color='b',linestyle='none')
    #align='center', width=width
    ax7.set_xscale("log", nonposx='clip')
    ax7.set_yscale("log", nonposy='clip')
    #f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/grain_freq_profile.txt', 'wb')
    #combined = np.vstack((x,y)).T
    #np.savetxt(f,combined,delimiter=" ", fmt="%s")
    #f.close()
    
    newvalues = [x for x in SelDicMap.grainList[grainID].ebsdGrain.misOriList if x != 0]
    
    
    hist, bins = np.histogram(np.arccos(newvalues) * 360 / np.pi, bins=100,range=[0,6],density=True)
    #hist=np.log10(hist*100)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    ax6.plot(center, hist, marker='x',color='b',linestyle='none')
    #align='center', width=width
    #ax6.set_xscale(nonposx='clip')
    #ax6.set_yscale(nonposy='clip')
    f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/irrdata.txt', 'wb')
    combined = np.vstack((center,hist)).T
    np.savetxt(f,combined,delimiter=" ", fmt="%s")
    f.close()
    
    
###### SHOW TEXT   
    plt.text(0.5, 0.5,'\n'.join([ str(myelement) for myelement in B ]), horizontalalignment='center',verticalalignment='center',transform=ax2.transAxes,fontsize=6)
    plt.tight_layout(pad=0)
    from builtins import print
    #print ('\n'.join([ str(myelement) for myelement in B ]))
    #print ('\n')
    #print("\n")

In [ ]:
angles=ebsdGrain.slipTraces
angles = [x * 180 / 3.141 for x in angles] 
print(angles)

### Detect slip system

In [ ]:
sel='ni'
acceptangle=5;

if sel=='i':
    ID=[0,1,4,9,10,15,16,19,20,22,23,23,25,28,32,33,34,36,36,39,40,40,43,44,44,48,48,54,56,56,57,57,59,60,63,65,66,67,67,68,68,69,70,72,73,74,75,78,78,79,80,81,82,83,85,86,88,89,91,92,94,96,99,100,101,103,104,105,106,106,107,109,114]
    DetAngles=[66.6,139.54,141.25,142.17,49.3,69.6,137.14,133.47,138.75,57.29,121.48,61.02,47.38,41.11,93,140.71,138.03,34.63,149.94,133.04,65.63,108.79,59.46,35.61,140.55,68.52,120.35,67.82,57.46,42.59,58.7,118.09,128.387,136.29,47.19,132.72,137,124.29,68.47,121.52,57.26,47.09,129.47,55.96,52.96,135.32,44.09,37.04,151.96,56.71,130.39,125.84,143.11,50.45,141.42,48.6,45.67,30.22,58.41,143.87,129.07,53.5,56.72,141.66,131.17,49.82,49.53,55.33,119.45,59.17,72.93,38.02,143.04]
    SelDicMap=DicMap
    print(len(DetAngles))

if sel=='ni':
    ID=[2,8,11,12,13,21,22,24,25,27,30,36,36,39,41,42,45,47,48,49,52,52,53,54,55,56,57,58,59,60,61,62,66,70,72,75,75,76,78,81,84,93,95,98,98,99,99,103,104,105,109,110,112,96]
    DetAngles=[58.86,150.9,51.55,139.17,132.75,160,53.27,80.04,134.59,148.31,142.35,35.42,152.84,57.58,126.01,44.15,120.89,123.87,121.69,74.34,32.94,151.83,53.17,45.67,51.5,131.95,64.81,119.54,159.72,146,130.51,128.56,145.64,130.18,38.35,116.87,67.55,50.78,53.13,127,131.52,66.05,134.08,59.16,112.88,130.27,69.69,142.85,111.33,124.76,37.18,53.78,50.1,101.01]
    SelDicMap=DicMap2
    print(len(DetAngles))

bigdevpr=[]; bigdevpy=[]; bigschmid=[]; bigsystem=[]; bigmean=[]; bigmax=[]; counter=[0,0,0,0,0,0]

for i in range (0,len(ID)):
    grainID=ID[i]
    ebsdGrain = SelDicMap[grainID].ebsdGrain
    ebsdGrain.calcSlipTraces()
    schdFacs = ebsdGrain.averageSchmidFactors
    slipTraces = ebsdGrain.slipTraces
    angles=[]; systems=[]; detected=[]; dev=[];
    systems=['Ba','Pr1','Pr2','Pr3','Py1','Py2','Py3','Py4','Py5','Py6']
    
### CALCULATE SLIP TRACE ANGLES FOR ALL SYSTEMS

    # Apply differential angle

    if sel=='i':
        diff = 2.4
    if sel=='ni':
        diff = 4
        
    angles = ebsdGrain.slipTraces
    angles = [(x * 180 / 3.141)+90+diff for x in angles] 

        
### COMPARE ALL SYSTEMS TO ALL ANGLES  
    
    for x in range(0,10):
        if angles[x] < 0: angles[x] += 180
        if angles[x] > 180: angles[x] -= 180
        if angles[x]-acceptangle <= DetAngles[i] <= angles[x]+acceptangle:
            detected.append(systems[x])
            dev.append(DetAngles[i]-angles[x])
            
### NONE DETECTED
    if len(dev)==0:
        counter[0]+=1
        bigsystem.append('None')
### ONE DETECTED
    if len(dev)==1:
        bigmean.append(np.mean(SelDicMap.grainList[i].maxShearList)*100)
        bigmax.append(np.max(SelDicMap.grainList[i].maxShearList)*100)
        counter[4]+=1
        if 'Ba' in detected[0]:
            counter[1]+=1
            bigsystem.append('Ba')
            bigschmid.append(np.max((ebsdGrain.averageSchmidFactors[0])))
        if 'Pr' in detected[0]:
            bigdevpr.append(np.float(dev[0]))
            counter[2]+=1
            bigsystem.append('Pr')
            if 'Pr1' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[1]))
            if 'Pr2' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[2]))
            if 'Pr3' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[3]))
        if 'Py' in detected[0]:
            bigdevpy.append(np.float(dev[0]))
            counter[3]+=1
            bigsystem.append('Py')
            if 'Py1' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[4]))
            if 'Py2' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[5]))
            if 'Py3' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[6]))
            if 'Py4' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[7]))
            if 'Py5' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[8]))
            if 'Py6' in detected[0]:
                bigschmid.append(np.max(ebsdGrain.averageSchmidFactors[9]))
                
    if len(dev)>1:
        
        systemconc = ''.join(map(str, detected))
        systemconcnonum = ''.join([i for i in systemconc if not i.isdigit()])
        
        bigsystem.append(systemconcnonum)
        
print("Basal: {0} ({3:.1f}%) / Prismatic: {1} ({4:.1f}%) / Pyramidal: {2} ({5:.1f}%)".format(counter[1],counter[2],counter[3],counter[1]*100/counter[4],counter[2]*100/counter[4],counter[3]*100/counter[4]))

print("One solution: {0} / No solution: {1} / Multiple: {2} / Total: {3}".format(counter[4],counter[0],len(ID)-counter[4]-counter[0],len(ID)))
print(np.mean(bigdevpr))

print("\nID\tAngle\tSS")
for c1, c2, c3 in zip(ID, bigsystem,DetAngles):
    print ('{0}\t{1}\t{2}'.format(c1, c3, c2))

plot = 1
if plot==1:
    hist, bins = np.histogram(bigdevpr, bins=20,range=[-5,5],density=False);
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
    hist2, bins2 = np.histogram(bigdevpy, bins=20,range=[-5,5],density=False);
    width2 = 0.7 * (bins[1] - bins[0])
    center2 = (bins[:-1] + bins[1:]) / 2
    plt.bar(center2, hist2, bottom=hist, align='center', width=width2)
    axes = plt.gca()
    axes.set_ylim([0,10])
    
if plot==2:
    hist, bins = np.histogram(bigschmid, bins=40,range=[0,0.5],density=False);
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
if plot == 3:
    fig=plt.scatter(bigschmid,bigmax)
    plt.xlim(0, 0.5)
    

### Plot profile across max shear map

In [ ]:
def plot_profile(def_map,start_point,end_point):
    x0,y0  = start_point[0], start_point[1] # These are in _pixel_ coordinates!!
    x1,y1 = end_point[0], end_point[1]
    profile_length=np.sqrt((y1-y0)**2+(x1-x0)**2)
    num = np.round(profile_length)
    x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)

    # Extract the values along the line, using cubic interpolation
    zi = scipy.ndimage.map_coordinates(np.transpose(def_map), np.vstack((x,y)))
    plt.figure(figsize=(20,6))
    gs = gridspec.GridSpec(1, 2, height_ratios=[1]) 
    ax0=plt.subplot(gs[0])
    ax1=plt.subplot(gs[1])
    ax0.imshow(def_map,vmin=-0.00,vmax=0.2,interpolation='bilinear',cmap='viridis');
    ax0.plot([x0, x1], [y0, y1], 'rx-',lw=2);
    ax0.axis('image');
    ax1.plot(zi)
    ax1.axis('tight')
    return zi;

In [ ]:
#corr_profile=plot_profile(DicMap.max_shear,[583,627],[445,772])
#corr_profile2=plot_profile(DicMap2.max_shear,[820,607],[911,718])
corr_profile2=plot_profile(DicMap.crop(DicMap2.max_shear),[160,426],[361,525])
#f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
#np.savetxt(f,corr_profile,fmt='%.6f')
#f.close()
f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
np.savetxt(f,corr_profile2,fmt='%.6f')
f.close()

### 2-point correlation GRAIN

In [ ]:
unzippedCoordlist = list(zip(DicMap.grainList[DicMap.currGrainId].coordList))
x0 = min(unzippedCoordlist[0])
y0 = min(unzippedCoordlist[1])
xmax = max(unzippedCoordlist[0])
ymax = max(unzippedCoordlist[1])
grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)

for coord, maxShear in zip(DicMap.grainList[DicMap.currGrainId].coordList, DicMap.grainList[DicMap.currGrainId].maxShearList):
            grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear

x=acorr_map(grainMaxShear)
plt.imshow(x,cmap='viridis')

### 2-point correlation MAP

In [ ]:
def acorr_map(def_map,c_range=[]):
    acorr=(np.fft.fft2(def_map)*np.conjugate(np.fft.fft2(def_map)))
    ashift=np.fft.fftshift(acorr)
    corr_map=np.log(np.fft.fftshift((np.abs(np.fft.ifft2(ashift)))))
    if c_range==[]:
        plt.imshow(corr_map, interpolation='nearest', cmap='viridis');
    else:
        plt.imshow(corr_map, interpolation='nearest', cmap='viridis',
                   vmin=c_range[0], vmax=c_range[1]);
    return corr_map

x=acorr_map(DicMap.max_shear[200:1200,200:1200])
y=acorr_map(DicMap2.max_shear[200:1200, 200:1200])

plt.imshow(DicMap.max_shear[200:1200,200:1200])

plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(y,cmap='viridis')
ax1.imshow(x,cmap='viridis')

plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(y[450:550,450:550],cmap='viridis')
ax1.imshow(x[450:550,450:550],cmap='viridis')

### Plot max shear for isolated grain with slip traces

In [ ]:
DicMap.grainList[DicMap.currGrainId].calcSlipTraces()
DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

#DicMap.grainList[DicMap.currGrainId].calcSlipTraces(correctAvOri=False)
#DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

In [ ]:
scale=0.02
fig = plt.figure()
plt.subplot(221)

fig = plt.imshow(DicMap.f11,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f11")

plt.subplot(222)

fig = plt.imshow(DicMap.f12,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f12")

plt.subplot(223)

fig = plt.imshow(DicMap.f21,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f21")

plt.subplot(224)

fig = plt.imshow(DicMap.f22,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f22")

plt.show()

### Display maps with homologous points

In [ ]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
fig, ((ax0, ax1, ax2)) = plt.subplots(1, 3, figsize=(20, 10))
#ax0 = plt.subplot(121)
#ax1 = plt.subplot(122, sharex=ax0, sharey=ax0)
#ax2 = plt.subplot(221, sharex=ax0, sharey=ax0)

ax0.imshow(-EbsdMap.boundaries, vmax=1, cmap='gray')

ax2.imshow(DicMap2.crop(DicMap.max_shear),vmin=0.0, vmax=0.1,cmap='viridis')
ax2.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)


rawImagePath = "irr1/irr_000.bmp"

rawImage = io.imread(rawImagePath)
ax1.imshow(DicMap.crop(rawImage))
ax1.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
shear_map_filt=DicMap2.f11[200:1200,200:1200]
shear_map_filt2=DicMap2.max_shear[200:1200,200:1200]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='viridis',vmin=-0.01,vmax=0.1)
ax1.imshow(shear_map_filt2,cmap='viridis',vmax=0.1)

### SPP MSS/Rot

In [ ]:
shear_map_filt=DicMap2.f21[540:610,590:670]-DicMap2.f12[540:610,590:670]
shear_map_filt2=DicMap2.max_shear[540:610,590:670]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='coolwarm',vmin=-0.08,vmax=0.1)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.00,vmax=0.08)
#ax0.title('max shear', fontsize=20)

#f11_profile=plot_profile(shear_map_filt2,[30,50],[44,66])
#f11_profile=plot_profile(shear_map_filt,[30,50],[44,66])
maxStrain = np.max(DicMap.max_shear)*100
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain)) 

In [ ]:
shear_map_filt=DicMap.f21[840:1210,890:1270]-DicMap.f12[840:1210,890:1270]
shear_map_filt2=DicMap.max_shear[840:1210,890:1270]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='coolwarm',vmin=-0.0008,vmax=0.001)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.0000,vmax=0.0008)
#ax0.title('max shear', fontsize=20)

#f11_profile=plot_profile(shear_map_filt2,[30,50],[44,66])
#f11_profile=plot_profile(shear_map_filt,[30,50],[44,66])
maxStrain = np.max(DicMap.max_shear)*100
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain)) 

In [ ]:
selDicMap = DicMap
shear_map_filt=((selDicMap.crop(selDicMap.f21)-selDicMap.crop(selDicMap.f12))/2)*(180/3.141)
shear_map_filt2=selDicMap.crop(selDicMap.max_shear*100)
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1], sharex=ax0, sharey=ax0)
ax0.imshow(shear_map_filt,cmap='coolwarm',vmin=-3,vmax=3)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0,vmax=8)
ax0.add_artist(ScaleBar(pixelinm))
ax1.add_artist(ScaleBar(pixelinm))

ax0.xaxis.set_visible(False)
ax0.yaxis.set_visible(False)
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)

In [ ]:
### Smaller

In [ ]:
cropmap=DicMap2.max_shear[200:1200,200:1200]
plt.figure()
gs = gridspec.GridSpec(1, 1)
ax0=plt.subplot(gs[0])
ax0.imshow(cropmap,cmap='coolwarm',vmin=-0.08,vmax=0.1)

maxStrain = np.max(cropmap)*100
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain)) 

In [ ]:
shear_map_filt=DicMap.f21[1000:1200,850:1200]-DicMap2.f12[1000:1200,850:1200]
shear_map_filt2=DicMap.f11[1000:1200,850:1200]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='viridis',vmin=0.00,vmax=0.08)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.00,vmax=0.08)

### Threshold

In [ ]:
np.percentile(DicMap.crop(DicMap.max_shear),95)

In [ ]:
#99th irr 19.05   nonirr 6.16
#98th irr 9.61 nonirr 5.37

threshold=0.0449589
shear_map_filt=DicMap2.crop(DicMap2.max_shear)>threshold

import scipy.misc
scipy.misc.imsave('newdata/nirr_threshold.bmp', DicMap2.crop(DicMap2.max_shear)>threshold)

plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
strain_title='Threshold: {:2.3f}'.format(threshold)
ax0.imshow(shear_map_filt,cmap='binary')
ax0.set_title(strain_title)
ax1.plot(profile_filt)

In [ ]:
threshold=0.051608277257999366
shear_map_filt=DicMap.crop(DicMap.max_shear)>threshold

import scipy.misc
scipy.misc.imsave('newdata/irr_threshold.bmp', DicMap.crop(DicMap.max_shear)>threshold)

plt.figure(figsize=(30,15))
gs = gridspec.GridSpec(1, 1)
ax0=plt.subplot(gs[0])
strain_title='Threshold: {:2.3f}'.format(threshold)
ax0.imshow(shear_map_filt,cmap='viridis')
ax0.set_title(strain_title)

### c Axis misori boundary map

In [ ]:
SelDicMap=DicMap2
SelDicMap.buildNeighbourNetwork()
import skimage as skimage

xmax, ymax = SelDicMap.xDim, SelDicMap.yDim
map = np.full((ymax,xmax), 0, dtype=float)   ### ->0

for i in SelDicMap.neighbourNetwork:
    for j in range(0,len(SelDicMap.neighbourNetwork.neighbors(i))):
        
        grainonedic=SelDicMap.grainList[i]
        graintwodic=SelDicMap.grainList[SelDicMap.neighbourNetwork.neighbors(i)[j]]

        ### GENERATE GRAIN ONE OUTLINE
        grainoneoutline = np.full((ymax,xmax), 0, dtype=int)
        for coord in grainonedic.coordList:
            grainoneoutline[coord[1], coord[0]] = 1
            grainoneoutline[coord[1]-1, coord[0]] = 1
            grainoneoutline[coord[1], coord[0]-1] = 1
            grainoneoutline[coord[1]-1, coord[0]-1] = 1
        
        ### GENERATE GRAIN TWO OUTLINE
        graintwooutline = np.full((ymax,xmax), 0, dtype=int)
        for coord in graintwodic.coordList:
            graintwooutline[coord[1], coord[0]] = 1
            graintwooutline[coord[1]-1, coord[0]] = 1
            graintwooutline[coord[1], coord[0]-1] = 1
            graintwooutline[coord[1]-1, coord[0]-1] = 1   
        
        #combinededges=edgesone&edgestwo
        combinededges=grainoneoutline&graintwooutline
        
        misori = Quat.misOri(grainonedic.ebsdGrain.refOri,graintwodic.ebsdGrain.refOri,symGroup='hexagonal')
        misori = 360 * np.arccos(misori) / np.pi

        cAxisVectorCrystal1 = Quat(grainonedic.ebsdGrain.refOri).transformVector(np.array([0, 0, 1]))
        cAxisVectorCrystal2 = Quat(graintwodic.ebsdGrain.refOri).transformVector(np.array([0, 0, 1]))
        
        dotted=cAxisVectorCrystal1.dot(cAxisVectorCrystal2)
        angle=np.arccos(dotted)*180/np.pi

        
        if angle<0:
            angle=angle+180
        if angle>90:
            angle=180-angle
        
        map[combinededges==1]=angle
        map2=skimage.morphology.dilation(map, selem=None)
        map2[map2==0]=np.nan
        
fig=plt.figure()
img=plt.imshow(map2,vmin=0,cmap='coolwarm')
plt.colorbar(img);

In [ ]:
print(angle)

### Rotate map

In [ ]:
def rotate_map(x_disp,y_disp,theta_deg):
    theta=theta_deg*np.pi/180
    x_prime=np.cos(theta)*x_disp-np.sin(theta)*y_disp
    y_prime=np.sin(theta)*x_disp+np.cos(theta)*y_disp
    return x_prime,y_prime

def get_rotated(xd,yd,rotation):
    rotated=rotate_map(xd,yd,theta_deg=rotation)
    xd_prime=rotated[0]
    yd_prime=rotated[1]
    f11_grad=np.gradient(xd_prime,6,6)[1]
    f12_grad=np.gradient(xd_prime,6,6)[0]
    f21_grad=np.gradient(yd_prime,6,6)[1]
    f22_grad=np.gradient(yd_prime,6,6)[0]  
    return f11_grad,f12_grad,f21_grad,f22_grad

def rotate_interactive(map_fig,xd,yd,component,rotation,colourmap='viridis',cmin=0.0,cmax=0.4):
    f_rotated=get_rotated(xd,yd,rotation)
    if component == 'f11':
        map_fig.set_data(f_rotated[0])
        map_fig.axes.set_title(r'$F_{11}$')
        print(np.mean(f_rotated[0]))
    if component == 'f12':
        map_fig.set_data((f_rotated[1]))
        map_fig.axes.set_title(r'$D_{12}$')
        print(np.mean(f_rotated[1]))
    if component == 'f21':
        map_fig.set_data(f_rotated[2])
        map_fig.axes.set_title(r'$F_{21}$')
        print(np.mean(f_rotated[2]))
    if component == 'f22':
        map_fig.set_data(f_rotated[3])
        map_fig.axes.set_title(r'$F_{22}$')
        print(np.mean(f_rotated[3]))
    if component == 'Shear':
        map_fig.set_data((f_rotated[1]+f_rotated[2])/2)
        map_fig.axes.set_title(r'$Shear$')
        print(np.mean((f_rotated[1]+f_rotated[2])/2))
    if component == 'theta3':
        map_fig.set_data(f_rotated[2]/f_rotated[0])
        map_fig.axes.set_title(r'$theta3$')
        print(np.mean(f_rotated[2]/f_rotated[0]))
    if component == 'omega3':
        map_fig.set_data(0.5*f_rotated[1]-0.5*f_rotated[2])
        map_fig.axes.set_title(r'$omega3$')
        print(np.mean(0.5*(f_rotated[1]-f_rotated[2])))
    map_fig.set_clim([cmin,cmax])
    map_fig.set_cmap(colourmap)
    map_fig.set_clim([cmin,cmax])
    plt.draw()
    
def rotate_print(xd,yd,component,rotation):
    f_rotated=get_rotated(xd,yd,rotation)
    if component == 'f11':
        print(f_rotated[0])
        print(np.mean(f_rotated[0]))
    if component == 'f12':
        print(f_rotated[1])
        print(np.mean(f_rotated[1]))
    if component == 'f21':
        print(f_rotated[2])
        print(np.mean(f_rotated[2]))
    if component == 'f22':
        print(f_rotated[3])
        print(np.mean(f_rotated[3]))

In [ ]:
xdprime,ydprime=rotate_map(DicMap.crop(DicMap.x_map),DicMap.crop(DicMap.y_map),66)
f11_grad=get_rotated(DicMap.crop(DicMap.x_map),DicMap.crop(DicMap.y_map),66)

In [ ]:
SelDicMap=DicMap2
fig,figax=plt.subplots(figsize=(10,8))
interactive_figure=figax.imshow(SelDicMap.crop(0.5*SelDicMap.f12-0.5*SelDicMap.f21),cmap='coolwarm',vmin=-0.0524,vmax=0.0524)

In [ ]:
rotate_interactive(map_fig=interactive_figure,xd=SelDicMap.crop(SelDicMap.x_map),yd=SelDicMap.crop(SelDicMap.y_map),
                   component='theta3',rotation=100,colourmap='coolwarm',cmin=-10,cmax=10)

### Schmid factor map

In [ ]:
import matplotlib as mpl
from skimage import morphology as mph

selDicMap=DicMap
percentile=95
fig = plt.figure(frameon=False, figsize=(8,7))
schmidlist=[];

ax = plt.Axes(fig, [0., 0., 0.9, 1.])
ax.set_axis_off()
fig.add_axes(ax)

cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['red', 'red'], 256)
cmap2._init()
cmap2._lut[:, -1] = np.linspace(0, 1, cmap2.N + 3)

avschmid = np.zeros([selDicMap.yDim, selDicMap.xDim])
for Grain in selDicMap.grainList:
    #max Schmid factor
    currentSchmidFactor = np.max([Grain.ebsdGrain.averageSchmidFactors[1],Grain.ebsdGrain.averageSchmidFactors[2],Grain.ebsdGrain.averageSchmidFactors[3]])
    schmidlist.append(currentSchmidFactor)       
    # currentSchmidFactor = grain.averageSchmidFactors[0][0]
    for coord in Grain.coordList:
        avschmid[coord[1], coord[0]] = currentSchmidFactor
prism=ax.imshow(avschmid,interpolation='none', cmap='gray', vmin=0, vmax=0.5)

boundariesImage = -selDicMap.boundaries
boundariesImage2 = mph.binary_dilation(boundariesImage)
img=ax.imshow(boundariesImage2, cmap=cmap1, vmin=0, vmax=1)

cbar=plt.colorbar(prism, ax=ax,fraction=0.03, pad=0.02, anchor=(0.0, 0.12))
cbar.set_label(label="Prismatic Schmid Factor",size=12)
cbar.ax.tick_params(labelsize=11) 

#img=ax.imshow(map2, cmap="Blues", vmin=0, vmax=90)
#cbar=plt.colorbar(img, ax=ax,fraction=0.03, pad=0.02, anchor=(0.0, 0.12))
#cbar.set_label(label="c-axis misorientation (°)",size=12)
#cbar.ax.tick_params(labelsize=11) 

threshold=np.percentile(selDicMap.crop(selDicMap.max_shear),percentile)
print(threshold)
thresholdmap=selDicMap.crop(selDicMap.max_shear)>threshold
ax.imshow(thresholdmap,cmap=cmap2,alpha=0.9)

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

fig.savefig('fig2.png',dpi=600, bbox_inches='tight', pad_inches=0)
print('Average prismatic schmid: {0:.3f} +- {1:.3f}'.format(np.mean(schmidlist),np.std(schmidlist)))

In [ ]:
print(groups)

In [ ]:
print(Grain.ebsdGrain.averageSchmidFactors[3])

In [ ]:
print(np.shape(DicMap.crop(DicMap.max_shear)))

In [ ]:
print(Grain.ebsdGrain.averageSchmidFactors[3])

In [ ]:
print(1152/64)

In [ ]:
import math
import matplotlib
import scipy.ndimage

cropped2=DicMap.crop(DicMap.max_shear)*100
cropped3=cropped2[0:1024, 0:1024]
print(np.shape(cropped3))

coarse_maps=[]
binnings=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

print("Res\t0.1\tMean\t99.9\tMax")
base=0.078125
for bins in binnings:
    
        one=np.shape(cropped3)[0]//bins
        two=np.shape(cropped3)[1]//bins
        big_view=cropped3.reshape(one,bins,two,bins)
        big_view2=big_view.mean(axis=3).mean(axis=1)
        
        big_view2=scipy.ndimage.zoom(big_view2, bins, order=0)
        
        coarse_maps.append(big_view2)
        
        print("{0:.0f}\t{1:.2f}\t{2:.2f}\t{3:.2f}\t{4:.2f}".format(bins*base*1000, np.percentile(big_view2,0.1), np.mean(big_view2), np.percentile(big_view2,99.9), np.max(big_view2)))

matplotlib.pyplot.imsave('1.png', coarse_maps[0], vmax=10)
matplotlib.pyplot.imsave('2.png', coarse_maps[1], vmax=10)
matplotlib.pyplot.imsave('4.png', coarse_maps[2], vmax=10)
matplotlib.pyplot.imsave('8.png', coarse_maps[3], vmax=10)
matplotlib.pyplot.imsave('16.png', coarse_maps[4], vmax=10)
matplotlib.pyplot.imsave('32.png', coarse_maps[5], vmax=10)
matplotlib.pyplot.imsave('64.png', coarse_maps[6], vmax=10)
matplotlib.pyplot.imsave('128.png', coarse_maps[7], vmax=10)
matplotlib.pyplot.imsave('256.png', coarse_maps[8], vmax=10)
matplotlib.pyplot.imsave('512.png', coarse_maps[9], vmax=10)
matplotlib.pyplot.imsave('1024.png', coarse_maps[10], vmax=10)

#from skimage import morphology as mph
#import matplotlib as mpl
#cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
#cmap1._init()
#cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
#boundariesImage = -DicMap.boundaries
#boundariesImage = mph.binary_dilation(boundariesImage)
#boundariesImage=boundariesImage[0:1024, 0:1024]
#matplotlib.pyplot.imsave('boundaryi.png', boundariesImage, cmap=cmap1, vmin=0, vmax=1)

In [ ]:
plt.figure()
plt.imshow(coarse_maps[5], vmax=10,cmap='viridis',interpolation='nearest')
plt.colorbar()

### Plot profile clicky

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt  
import matplotlib as mpl
import skimage.morphology as mph
from lmfit.models import GaussianModel

class LineSlice:

    def __init__(self, data, img, ax, fig):

        self.img = img
        self.fig = fig
        self.ax = ax
        self.data = data

        # register the event handlers:
        self.cidclick = img.figure.canvas.mpl_connect('button_press_event', self)
        self.cidrelease = img.figure.canvas.mpl_connect('button_release_event', self)

        self.markers, self.arrow = None, None   # the lineslice indicators on the pcolormesh plot
        self.line = None    # the lineslice values plotted in a line

    def __call__(self, event):
        '''Matplotlib will run this function whenever the user triggers an event on our figure'''
        if self.img.figure.canvas.manager.toolbar._active is not None: return   # exit if pyplot toolbar (zooming etc.) is active

        if event.name == 'button_press_event':
            self.p1 = (event.xdata, event.ydata)    # save 1st point
        elif event.name == 'button_release_event':
            self.p2 = (event.xdata, event.ydata)    # save 2nd point
            self.drawLineSlice()    # draw the Line Slice position & data

    def drawLineSlice( self ):
        
        x0,y0 = self.p1[0], self.p1[1]  # get user's selected coordinates
        x1,y1 = self.p2[0], self.p2[1]
        length = int( np.hypot(x1-x0, y1-y0) )
        x, y = np.linspace(x0, x1, length), np.linspace(y0, y1, length)
        zi = scipy.ndimage.map_coordinates(np.transpose(self.data), np.vstack((x,y)))

        if self.markers != None:
            if isinstance(self.markers, list):
                self.markers[0].remove()
            else:
                self.markers.remove()
        if self.arrow != None:
            self.arrow.remove()

        self.markers = self.img.axes.plot([x0, x1], [y0, y1], 'wo')   

        self.arrow = self.img.axes.annotate("",
                    xy=(x0, y0), xycoords='data',
                    xytext=(x1, y1), textcoords='data',
                    arrowprops=dict(
                        arrowstyle="<-",connectionstyle="arc3",color='white',alpha=0.7,linewidth=3),
                    )

        # plot the data along the line on provided `ax`:
        if self.line != None:
            self.line[0].remove()   # delete the plot
        self.line = self.ax.plot(zi)
        
        indexes=peakutils.indexes(zi,thres=4/np.max(zi))
        #indexes=peakutils.indexes(zi,thres=0.1)
        print('Indexes:')
        print(indexes)
        print('Height:')
        print(zi[indexes])
        
        fwhms=[]
        for index in indexes:
            side=5
            ydata = np.array(zi[index-side:index+side])
            xdata = np.arange(side*2)
    
            gmodel = GaussianModel()
            params = gmodel.make_params(amp=ydata.max(), cen=xdata.mean(), sig=xdata.std())
            result = gmodel.fit(ydata, params, x=xdata)
            params['amplitude'].vary = False
            params['center'].vary = False

            dict1=result.params.valuesdict()
            self.ax.scatter(xdata+index-side,ydata)
            self.ax.plot(xdata+index-side, result.best_fit, 'r-')
            fwhms.append(6*dict1['sigma'])
        

        print(fwhms)
        for index in indexes:
            self.peaks=self.ax.axvline(x=index)

        plt.gca().set_ylim(bottom=0)
        self.fig.canvas.draw()
        
        np.savetxt('line.csv', (zi), delimiter='\n')
        
fig, (ax1, ax2) = plt.subplots(figsize=(16,10),nrows=2, ncols=1)
img = ax1.imshow(DicMap.crop(DicMap.max_shear)*100,vmax=10)


cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
boundariesImage = -DicMap.boundaries
boundariesImage2 = mph.binary_dilation(boundariesImage)
img=ax1.imshow(boundariesImage2, cmap=cmap1, vmin=0, vmax=1)

LnTr = LineSlice(DicMap.crop(DicMap.max_shear)*100, img, ax2, fig) 

In [ ]:
DicMap.calcProxigram(numTrials=100)
DicMap2.calcProxigram(numTrials=100)

### Proxigram

In [ ]:
plt.figure(figsize=(18,8))
plt.imshow(DicMap.proxigramArr, vmax=140)

In [ ]:
m = np.ma.masked_outside(DicMap.proxigramArr,0,10)
new_x = np.ma.masked_where(np.ma.getmask(m), DicMap.crop(DicMap.max_shear))
plt.imshow(new_x,vmin=0,vmax=0.1)

In [ ]:
print('irr')
for x in [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100,105,110,115,120,125,130,135,140]:
    y=x+5
    m = np.ma.masked_outside(DicMap.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap.crop(DicMap.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.mean(new_x),np.std(new_x)))

In [ ]:
print('irr')
for x in [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140]:
    y=x+10
    m = np.ma.masked_outside(DicMap.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap.crop(DicMap.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.mean(new_x),np.std(new_x)))

In [ ]:
print('irr')
for x in [0,20,40,60,80,100,120,140]:
    y=x+20
    m = np.ma.masked_outside(DicMap.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap.crop(DicMap.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.mean(new_x),np.std(new_x)))

In [ ]:
print('irr')
for x in [0,20,40,60,80,100,120,140]:
    y=x+20
    m = np.ma.masked_outside(DicMap.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap.crop(DicMap.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.ma.MaskedArray.sum(new_x>0.0516),np.ma.MaskedArray.sum(new_x>0.0516)/np.ma.MaskedArray.count(m)))


In [ ]:
print('nirr')
for x in [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100,105,110,115,120,125,130,135,140]:
    y=x+5
    m = np.ma.masked_outside(DicMap2.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap2.crop(DicMap2.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.mean(new_x),np.std(new_x)))

In [ ]:
print('nirr')
for x in [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140]:
    y=x+10
    m = np.ma.masked_outside(DicMap2.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap2.crop(DicMap2.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.mean(new_x),np.std(new_x)))

In [ ]:
print('nirr')
for x in [0,20,40,60,80,100,120,140]:
    y=x+20
    m = np.ma.masked_outside(DicMap2.proxigramArr,x,y)
    new_x = np.ma.masked_where(np.ma.getmask(m), DicMap2.crop(DicMap2.max_shear))
    print('{}\t{}\t{}\t{}\t{}'.format(x,y,np.ma.MaskedArray.count(m),np.mean(new_x),np.std(new_x)))